In [6]:
import numpy as np
import csv
import matplotlib.pyplot as plt

In [7]:
# dummy mtrix
# Define the matrix
matrix = [
    [0, 0.5, 0, 0.1, 0.2, 0.7, 0, 0.3],
    [0.5, 0, 0.7, 0.3, 0.4, 0, 0.4, 0.2],
    [0, 0.7, 0, 0.2, 0, 0.6, 0, 0],
    [0.1, 0.3, 0.2, 0, 0, 0, 0, 0.7],
    [0.2, 0.4, 0, 0, 0, 0.2, 0, 0.5],
    [0.7, 0, 0.6, 0, 0.2, 0, 0.1, 0],
    [0, 0.4, 0, 0, 0, 0.1, 0, 0.3],
    [0.3, 0.2, 0, 0.7, 0.5, 0, 0.3, 0]
]

# Save the matrix into a CSV file
with open('/home/gabridele/Desktop/symmetric_matrix.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for row in matrix:
        writer.writerow(row)

In [8]:
path_dummy_mtrix = '/home/gabridele/Desktop/symmetric_matrix.csv'
path_to_mtrx = '/home/gabridele/Desktop/connectome_sub-100610.csv'
#path_to_mtrx = '/Users/gabrieledele/Desktop/Hub_identification/gabriele_sc_connectomes/connectome_sub-100610.csv'
matrix = np.loadtxt(path_dummy_mtrix, delimiter=',')

#resized_matrix = original_matrix[:100, :100]
#print(resized_matrix.shape)

In [9]:
# Load the connectivity matrix from a CSV file
def load_connectivity_matrix(file_path):
    return np.loadtxt(file_path, delimiter=',')

In [10]:
# Set the activation threshold
## che non ci siano righe/cols con zero
## to each subj their threshold
## far andare simulazione

In [11]:
# Select seed nodes randomly
def select_seed_nodes(num_nodes, num_seeds):
    return np.random.choice(num_nodes, num_seeds, replace=False)

In [23]:
def find_highest_threshold(connectivity_matrix, max_iterations=10):
    num_nodes = connectivity_matrix.shape[0] # so 8x8 in my dummy case
    successful_thresholds = []  # List to store successful threshold values
    weighted_degree = np.sum(connectivity_matrix, axis=0)
    lowest_weighted_degree = np.min(weighted_degree)
    print("Lowest weighted degree:", lowest_weighted_degree)
    index_lowest_degree = np.argmin(weighted_degree)
    print("with index:", index_lowest_degree)
    seed_nodes = index_lowest_degree

    
    # Initialize active nodes. creates empty array except for
    active_nodes = np.zeros(num_nodes, dtype=bool)
    active_nodes[seed_nodes] = True
    
    # Initialize threshold
    threshold = 0.0

    # Initialize iteration counter
    iteration = 0

    while not np.all(active_nodes) and iteration < max_iterations: # Checks if there are still inactive nodes
        new_active_nodes = np.zeros(num_nodes, dtype=bool) # create zeroes-array to store activated nodes

        # Set threshold for this iteration
        #if iteration == 0:
        #    threshold = 0.0  # Set initial threshold

        # Iterate over each node
        for node_idx in range(num_nodes):
            print(f"Current node: {node_idx+1}") # prints current node
            if not active_nodes[node_idx]:
                neighbors = connectivity_matrix[node_idx] # Retrieves row corresponding to the current node, aka neighbors
                neighbors_activated = active_nodes[np.where(neighbors > 0)[0]]
                degree_of_node = np.sum(connectivity_matrix[node_idx] > 0)

                # Calculate activation percentage
                activation_percentage = np.sum(neighbors_activated) / degree_of_node * 100

                # Update threshold if necessary
                if activation_percentage > threshold:
                    threshold = activation_percentage

                activation_condition = threshold < activation_percentage
                if activation_condition:
                    new_active_nodes[node_idx] = True

        # Update active nodes
        active_nodes = np.logical_or(active_nodes, new_active_nodes)

        iteration += 1
        print(f"Iteration {iteration}: Number of active nodes = {np.sum(active_nodes)}")

    # Check if all nodes are activated
    if np.all(active_nodes):
        print(f"Successful simulation with threshold: {threshold}%")
        successful_thresholds.append(threshold)

# Return the highest threshold allowing simulation completion
if successful_thresholds:
    highest_threshold = max(successful_thresholds)
    print("Highest threshold allowing simulation completion:", highest_threshold)
    return highest_threshold
else:
    print("Unable to find a threshold allowing simulation completion.")
    return None


In [15]:
# simulation function
def simulate_activation(connectivity_matrix, seed_nodes, activation_threshold, max_iterations):
    num_nodes = connectivity_matrix.shape[0]
    #print('num_nodes:', num_nodes)
    #print('seed_nodes:', seed_nodes)
    active_nodes = np.zeros(num_nodes, dtype=bool)
    #print('active_nodes:', active_nodes) 
    # Everything should be false, because created a new array with same num nodes and zeroed out, except for the seeds
    active_nodes[seed_nodes] = True
    #print('active_nodes2:', active_nodes[seed_nodes])
    #print('active_nodes3:', active_nodes)
    
    #active_nodes_history = []
    iteration = 0  # Initialize iteration counter
    
    while not np.all(active_nodes) and iteration < max_iterations: # Checks if there are still inactive nodes. Continues till all nodes are activated
        new_active_nodes = np.zeros(num_nodes, dtype=bool) # Array to store nodes that become activated in current iteration
        # Initially all nodes are inactive
        for node_idx in range(num_nodes): # Iterates over each node in the network
            if not active_nodes[node_idx]: #  Checks if current node is inactive
                neighbors = connectivity_matrix[node_idx] # Retrieves row corresponding to the current node, aka neighbors
                # Selects activated neighbors of current node by filtering active_nodes array based on non-zero entries in neighbors row
                neighbors_activated = active_nodes[np.where(neighbors > 0)[0]]
                degree_of_node = np.sum(connectivity_matrix[node_idx] > 0)  # Degree of the node by counting non-zero entries in row
                # Check if the number of activated neighbors exceeds the threshold
                # Threshold should be highest values that allows completion of cascade
                activation_condition = np.sum(neighbors_activated) / degree_of_node * 100 > activation_threshold
                if activation_condition:
                    # If so, then node becomes activated
                    new_active_nodes[node_idx] = True
        # Updates active_nodes array by performing a logical OR operation with new_active_nodes array 
        # Activated nodes from the current iteration are added to the overall set of activated nodes          
        active_nodes = np.logical_or(active_nodes, new_active_nodes)

        # Append the indices of active nodes to the history list
        #active_node_indices = np.where(active_nodes)[0]
        #active_nodes_history.append(active_node_indices)

        iteration += 1
        print(f"Iteration {iteration}: Number of active nodes = {np.sum(active_nodes)}")
        #save into array

     # Determine the maximum length of active nodes history
    #max_length = max(len(history) for history in active_nodes_history)

    # Pad the shorter arrays with a placeholder value to make them all the same length
    #padded_history = [np.pad(history, (0, max_length - len(history)), constant_values=-1) for history in active_nodes_history]

    # Convert the list of padded active nodes history into a NumPy array and transpose it
    #active_nodes_history_array = np.array(padded_history).T

    print("Simulation complete.")
    return active_nodes #, active_nodes_history_array

In [24]:
# Main function
def main():
    # Load the connectivity matrix
    connectivity_matrix = matrix
    
    # Select seed nodes
    num_nodes = connectivity_matrix.shape[0]
    num_seeds = 1

    # Find the highest threshold allowing simulation completion
    #threshold_values = np.arange(0, 0.01, 0.0001).tolist()  # Generate threshold values
    highest_threshold = find_highest_threshold(connectivity_matrix, max_iterations=10)
    print('yo mama:', highest_threshold)
    #highest_threshold = 0.0064
    '''for _ in range(1):
        seed_nodes = select_seed_nodes(num_nodes, num_seeds)
        active_nodes = simulate_activation(connectivity_matrix, seed_nodes, highest_threshold, max_iterations=10)
        print("Final activation state with highest threshold:", active_nodes)
    print('For loop complete')'''

    '''for node in active_nodes:
        print(node)'''
    ### Adjust later
    # Plot the results (Iterations vs. Number of Active Nodes)
    #plt.figure(figsize=(8, 6))
    #plt.plot(range(len(activated_nodes_counts)), activated_nodes_counts, marker='o', color='red')
    #plt.xlabel('Iterations')
    #plt.ylabel('Number of Active Nodes')
    #plt.title('Progress of Activation')
    #plt.grid(True)
    #plt.show()
    

if __name__ == "__main__":
    main()

Lowest weighted degree: 0.8
with index: 6
Current node: 1
Iteration 1: Number of active nodes = 1
Iteration 2: Number of active nodes = 1
Iteration 3: Number of active nodes = 1
Iteration 4: Number of active nodes = 1
Iteration 5: Number of active nodes = 1
Iteration 6: Number of active nodes = 1
Iteration 7: Number of active nodes = 1
Iteration 8: Number of active nodes = 1
Iteration 9: Number of active nodes = 1
Iteration 10: Number of active nodes = 1
Current node: 2
Iteration 1: Number of active nodes = 1
Iteration 2: Number of active nodes = 1
Iteration 3: Number of active nodes = 1
Iteration 4: Number of active nodes = 1
Iteration 5: Number of active nodes = 1
Iteration 6: Number of active nodes = 1
Iteration 7: Number of active nodes = 1
Iteration 8: Number of active nodes = 1
Iteration 9: Number of active nodes = 1
Iteration 10: Number of active nodes = 1
Current node: 3
Iteration 1: Number of active nodes = 1
Iteration 2: Number of active nodes = 1
Iteration 3: Number of activ